In [2]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
import tf_keras

In [3]:
devices = tf.config.list_physical_devices()
print("\nDevices: ", devices)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  details = tf.config.experimental.get_device_details(gpus[0])
  print("GPU details: ", details)


Devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU details:  {'device_name': 'METAL'}


In [4]:
# print("Tensorflow Verions: {}\n,\nGPU : {}".format(tf.__version__,tf.config.list_physical_devices('GPU')))

In [5]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # Disable GPU

In [6]:
IMAGE_CUT_DIR = "dental-dataset/images_cut"
LABEL_CUT_DIR = "dental-dataset/labels_cut"

In [7]:
image_cut_path = sorted(glob(os.path.join(IMAGE_CUT_DIR,"*.png")))
label_cut_path = sorted(glob(os.path.join(LABEL_CUT_DIR,"*.png")))

In [8]:
# for i in range(2):
#     plt.figure(figsize=(8,4))

#     plt.subplot(1,2,1)
#     plt.imshow(cv2.imread(image_cut_path[i]),cmap='gray')
#     plt.title("X-ray Image")
    
#     plt.subplot(1,2,2)
#     plt.imshow(cv2.imread(label_cut_path[i]),cmap='gray')
#     plt.title("Segmentation Mask (Caries)")

#     plt.show()

In [9]:
assert len(image_cut_path) == len(label_cut_path), "Mismatch between images and masks"

In [10]:
train_images, val_images, train_label, val_label = train_test_split(image_cut_path,label_cut_path,test_size=0.2,random_state=42)

In [11]:
train_images[1]

'dental-dataset/images_cut/1092.png'

In [12]:
# Preserving aspect ratio is more important in segmentation than in classification because we want object locations to remain accurate.
IMG_WIDTH = 256
IMG_HEIGHT = 128

def load_img_and_mask(image_path,label_path):
    
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image,channels=3)
    image = tf.image.resize(image,[IMG_HEIGHT,IMG_WIDTH])
    image = tf.cast(image, tf.float32) / 255.0  # Normalize

    labeled_image = tf.io.read_file(label_path)
    labeled_image = tf.image.decode_png(labeled_image,channels=1)
    labeled_image = tf.image.resize(labeled_image,[IMG_HEIGHT,IMG_WIDTH],method="nearest")
    labeled_image = tf.cast(labeled_image > 0, tf.uint8)  # Normalize

    return image,labeled_image
    
    

In [13]:
BATCH_SIZE = 1
def create_dataset(data):
    images,labels = data
    dataset = tf.data.Dataset.from_tensor_slices((images,labels))
    dataset = dataset.map(load_img_and_mask,num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=5)
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset
    

In [14]:
train_dataset = create_dataset((train_images,train_label))

2025-04-22 18:38:46.418041: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-04-22 18:38:46.418058: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-04-22 18:38:46.418062: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-04-22 18:38:46.418079: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-22 18:38:46.418087: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
val_dataset = create_dataset((val_images,val_label))

In [16]:
# for images, masks in train_dataset.take(1):
#     print("Image batch shape:", images.shape[0])
#     print("Mask batch shape:", masks.shape[0])
#     plt.subplot(2,2,1)
#     plt.imshow(tf.keras.utils.array_to_img(images[0]))
#     plt.subplot(2,2,2)
#     plt.imshow(tf.keras.utils.array_to_img(masks[0]))
    

In [17]:
def build_unet(input_shape):
    inputs = tf.keras.Input(input_shape)

    # Encoder
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    # Bottleneck
    b1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p2)
    b1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(b1)

    # Decoder
    u1 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(b1)
    u1 = layers.concatenate([u1, c2])
    c3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u1)
    c3 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c3)

    u2 = layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c3)
    u2 = layers.concatenate([u2, c1])
    c4 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(u2)
    c4 = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(c4)

    # Output
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c4)

    model = model.Model(inputs, outputs)
    return model


In [18]:
model = build_unet(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
model.summary()

UnboundLocalError: local variable 'model' referenced before assignment

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
EPOCHS = 5

trained_model = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=EPOCHS)

In [ ]:
trained_model.save('models/dental-segmentation2.keras')

In [ ]:
def plot_acurracy(stats):
    epochs = range(1, len(stats['loss']) + 1)

    
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, stats["accuracy"], label = "Train accuracy")
    plt.plot(epochs, stats["val_accuracy"], label = "Val accuracy")
    plt.legend(loc='upper right')
    plt.title('Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs, stats["loss"], label = "Train Loss")
    plt.plot(epochs, stats["val_loss"], label = "Val Loss")
    plt.legend(loc='upper right')
    plt.title('Loss')
    
    plt.show()    
    

In [ ]:
# plot_acurracy(trained_model.history)

In [ ]:
# loaded_model = tf_keras.models.load_model("models/dental-segmentation.keras")

In [ ]:
def display_prediction(image, ground_truth_mask, predicted_mask):
    """Helper to show image, actual mask, predicted mask side by side"""
    plt.figure(figsize=(12, 4))

    # Original image
    plt.subplot(1, 3, 1)
    plt.title("Input Image")
    plt.imshow(image)
    plt.axis("off")

    # Ground truth mask
    plt.subplot(1, 3, 2)
    plt.title("True Mask")
    plt.imshow(ground_truth_mask[:, :, 0], cmap="gray")
    plt.axis("off")

    # Predicted mask
    plt.subplot(1, 3, 3)
    plt.title("Predicted Mask")
    plt.imshow(predicted_mask[:, :, 0], cmap="gray")
    plt.axis("off")

    plt.show()

def predict_and_visualize(model, image, true_mask):
    """Runs prediction and visualizes the result"""
    image = tf.expand_dims(image, axis=0)  # (1, height, width, 3)
    prediction = model.predict(image)[0]   # Remove batch dimension
    
    # Threshold predicted mask to make it binary
    predicted_mask = tf.where(prediction > 0.5, 1.0, 0.0).numpy()

    display_prediction(image[0].numpy(), true_mask.numpy(), predicted_mask)

In [ ]:
# for i, (image, mask) in enumerate(val_dataset.take(2)):
#     print(f"Prediction for validation image #{i + 1}")
#     predict_and_visualize(model, image, mask)